In [1]:
import MetaTrader5 as mt5
from datetime import datetime
import pandas as pd
import time 
import numpy as np

In [2]:
# String inicial
base_string = 'BOVA'

# Gerar a lista com as variações de 'VALEA' até 'VALEZ'
result = [base_string + chr(i) for i in range(ord('A'), ord('Z') + 1)]

# Exibir o resultado
print(result[23])

BOVAX


In [4]:
mt5.initialize()

mt5.login(12712431,'Hammstein08*', 'Rico-PRD')
lista_opcoes = []


for nome in result:
    ticker = mt5.symbols_get(nome)
    print(len(ticker))
    for j in range(1,len(ticker)):
        lista_opcoes.append(ticker[j].name)

lista_opcoes[5]

24
24
29
10
9
17
375
382
308
131
68
77
24
24
29
10
9
17
375
382
308
131
68
77
0
0


'BOVAA128'

In [5]:
for i in range(len(lista_opcoes)):
    ticker = lista_opcoes[i]
    
    # Obter as informações do símbolo de uma vez
    mt5.symbol_select(ticker,True)

time.sleep(10)
    

In [56]:
dicionario_opcoes = {}
for i in range(len(lista_opcoes)):
    ticker = lista_opcoes[i]
    
    # Obter as informações do símbolo de uma vez
    
    symbol_info = mt5.symbol_info(ticker)
    ativo_base = mt5.symbol_info(ticker).basis
    mt5.symbol_select(ativo_base,True)
    if (ativo_base == ''):
        cota = 0
    else:
        cota = mt5.symbol_info(ativo_base).last


    if symbol_info is not None:
        # Construir o valor do dicionário com as informações
        dicionario_opcoes[ticker] = {
            'Last': symbol_info.last,
            'Bid': symbol_info.bid,
            'Ask': symbol_info.ask,
            'Volume': symbol_info.session_volume,
            'Data': datetime.fromtimestamp(symbol_info.time).strftime('%Y-%m-%d'),
            'Vencimento': datetime.fromtimestamp(symbol_info.expiration_time).strftime('%Y-%m-%d'),
            'Strike': symbol_info.option_strike,
            'Cotacao_Ativo' : cota,
            'Tipo': symbol_info.option_right,
            # 'Vencimento': symbol_info.expiration_time,
            'Abertura': symbol_info.session_open
        }

# Exibir o dicionário para conferir
print(dicionario_opcoes)

grade_opcoes = pd.DataFrame(dicionario_opcoes).T

opcoes_negociadas = grade_opcoes[grade_opcoes['Volume'] != 0]
opcoes_negociadas.loc[opcoes_negociadas['Tipo'] == 0, 'Tipo'] = 'Call'
opcoes_negociadas.loc[opcoes_negociadas['Tipo'] == 1, 'Tipo'] = 'Put'

df_opcoes = opcoes_negociadas.sort_values(by='Vencimento')

{'BOVAA100': {'Last': 0.0, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 0.0, 'Data': '1969-12-31', 'Vencimento': '2026-01-16', 'Strike': 10.0, 'Cotacao_Ativo': 135.32, 'Tipo': 0, 'Abertura': 0.0}, 'BOVAA119': {'Last': 0.0, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 0.0, 'Data': '2025-07-09', 'Vencimento': '2027-01-15', 'Strike': 119.0, 'Cotacao_Ativo': 135.32, 'Tipo': 0, 'Abertura': 0.0}, 'BOVAA125': {'Last': 21.87, 'Bid': 10.0, 'Ask': 0.0, 'Volume': 0.0, 'Data': '2025-07-09', 'Vencimento': '2026-01-16', 'Strike': 125.0, 'Cotacao_Ativo': 135.32, 'Tipo': 0, 'Abertura': 0.0}, 'BOVAA126': {'Last': 0.0, 'Bid': 0.01, 'Ask': 0.0, 'Volume': 0.0, 'Data': '2025-07-09', 'Vencimento': '2026-01-16', 'Strike': 126.0, 'Cotacao_Ativo': 135.32, 'Tipo': 0, 'Abertura': 0.0}, 'BOVAA127': {'Last': 0.0, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 0.0, 'Data': '1969-12-31', 'Vencimento': '2026-01-16', 'Strike': 127.0, 'Cotacao_Ativo': 135.32, 'Tipo': 0, 'Abertura': 0.0}, 'BOVAA128': {'Last': 0.0, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 0.0, '

C:\Users\eduardomoreira\AppData\Local\Temp\ipykernel_21180\3965053762.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opcoes_negociadas.loc[opcoes_negociadas['Tipo'] == 0, 'Tipo'] = 'Call'
C:\Users\eduardomoreira\AppData\Local\Temp\ipykernel_21180\3965053762.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opcoes_negociadas.loc[opcoes_negociadas['Tipo'] == 1, 'Tipo'] = 'Put'


In [43]:
df_opcoes

,Last,Bid,Ask,Volume,Data,Vencimento,Strike,Cotacao_Ativo,Tipo,Abertura
BOVAS130W2,0.03,0.02,0.03,87890.0,2025-07-09,2025-07-11,130.0,135.53,Put,0.02
BOVAG129W2,6.48,6.67,6.74,600.0,2025-07-09,2025-07-11,129.0,135.53,Call,6.54
BOVAG128W2,7.45,7.66,7.73,1600.0,2025-07-09,2025-07-11,128.0,135.53,Call,7.47
BOVAG127W2,9.29,8.66,8.73,47105.0,2025-07-09,2025-07-11,127.0,135.53,Call,9.27
BOVAS166W2,28.03,0.0,0.0,1000.0,2025-07-09,2025-07-11,166.0,135.53,Put,28.03
...,...,...,...,...,...,...,...,...,...,...
BOVAL2,4.55,3.01,15.0,1.0,2025-07-08,2026-12-18,180.0,135.53,Call,4.55
BOVAX159,12.54,0.13,13.0,2.0,2025-07-09,2026-12-18,160.0,135.54,Put,12.5
BOVAX13,3.48,3.47,4.5,391.0,2025-07-08,2026-12-18,130.0,135.54,Put,3.48
BOVAD175,9.96,1.05,0.0,7600.0,2025-07-08,2027-04-16,175.0,135.53,Call,10.5


In [60]:
import numpy as np
import pandas as pd
from scipy.stats import norm

def calcular_gregas_blackscholes(df, vol=0.15, taxa_r=0.10):
    df = df.copy()

    # Converte colunas de data
    df['Data'] = pd.to_datetime(df['Data'], errors='coerce')
    df['Vencimento'] = pd.to_datetime(df['Vencimento'], errors='coerce')

    # Remove linhas inválidas
    df = df.dropna(subset=['Data', 'Vencimento', 'Cotacao_Ativo', 'Strike'])

    # Tempo até o vencimento em anos (base 252 dias úteis)
    df['Dias_para_Venc'] = (df['Vencimento'] - df['Data']).dt.days
    df['T'] = df['Dias_para_Venc'] / 252

    # Evita divisão por zero ou log de zero
    df = df[df['T'] > 0]
    df = df[df['Cotacao_Ativo'] > 0]
    df = df[df['Strike'] > 0]

    # Extrai variáveis
    S = df['Cotacao_Ativo'].astype(float)
    K = df['Strike'].astype(float)
    T = df['T'].astype(float)
    r = taxa_r
    sigma = vol

    # Calcula d1 e d2
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    # Valor justo
    df['BS_Preco'] = np.where(
        df['Tipo'] == 'Call',
        S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2),
        K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    )

    # Gregas
    df['Delta'] = np.where(
        df['Tipo'] == 'Call',
        norm.cdf(d1),
        -norm.cdf(-d1)
    )
    df['Gamma'] = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    df['Vega'] = S * norm.pdf(d1) * np.sqrt(T)   # por 1% de vol
    df['Theta'] = np.where(
        df['Tipo'] == 'Call',
        (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)) / 252,
        (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2)) / 252
    )
    df['Rho'] = np.where(
        df['Tipo'] == 'Call',
        K * T * np.exp(-r * T) * norm.cdf(d2) / 100,
        -K * T * np.exp(-r * T) * norm.cdf(-d2) / 100
    )
    df['Delta'] = round(df['Delta'],4)
    df['Gamma'] = round(df['Gamma'],4)
    df['Vega'] = round(df['Vega'],4)
    df['Theta'] = round(df['Theta'],4)
    df['Rho'] = round(df['Rho'],4)
    df['BS_Preco'] = round(df['BS_Preco'],4)

    return df



In [63]:
df_comp = calcular_gregas_blackscholes(df_opcoes, vol = 0.1342,taxa_r=0.1490)

df_comp['UpSide'] = df_comp['Last'] - df_comp['BS_Preco']

df_comp.loc[(df_comp['Dias_para_Venc'] >=15) & (df_comp['Dias_para_Venc'] <= 45) ]

,Last,Bid,Ask,Volume,Data,Vencimento,Strike,Cotacao_Ativo,Tipo,Abertura,Dias_para_Venc,T,BS_Preco,Delta,Gamma,Vega,Theta,Rho,UpSide
BOVAG124W4,12.89,12.11,12.19,3320.0,2025-07-09,2025-07-25,124.0,135.32,Call,13.03,16,0.063492,12.4902,0.9980,0.0014,0.2150,-0.0734,0.0778,0.3998
BOVAG127W4,9.99,9.17,9.25,200.0,2025-07-09,2025-07-25,127.0,135.32,Call,9.99,16,0.063492,9.5402,0.9851,0.0082,1.2826,-0.0786,0.0786,0.4498
BOVAS135W4,1.12,1.12,1.16,501.0,2025-07-09,2025-07-25,135.0,135.32,Put,0.98,16,0.063492,1.1290,-0.3569,0.0815,12.7185,-0.0241,-0.0314,-0.009
BOVAG126W4,10.34,10.15,10.22,1350.0,2025-07-09,2025-07-25,126.0,135.32,Call,10.45,16,0.063492,10.5187,0.9920,0.0048,0.7509,-0.0763,0.0785,-0.1787
BOVAS134W4,0.8,0.79,0.83,2305.0,2025-07-09,2025-07-25,134.0,135.32,Put,0.76,16,0.063492,0.8028,-0.2788,0.0734,11.4531,-0.0253,-0.0245,-0.0028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BOVAH140W4,2.11,1.0,0.0,274.0,2025-07-09,2025-08-22,140.0,135.32,Call,2.21,44,0.174603,2.5274,0.4545,0.0522,22.4109,-0.0690,0.1030,-0.4174
BOVAT130W4,0.1,0.05,0.0,2500.0,2025-07-09,2025-08-22,130.0,135.32,Put,0.1,44,0.174603,0.4297,-0.1137,0.0254,10.8852,-0.0073,-0.0276,-0.3297
BOVAH136W4,4.25,1.95,0.0,8.0,2025-07-09,2025-08-22,136.0,135.32,Call,4.23,44,0.174603,4.6093,0.6564,0.0485,20.8019,-0.0815,0.1470,-0.3593
BOVAT124W4,0.25,0.0,0.0,2.0,2025-07-08,2025-08-22,124.0,135.32,Put,0.16,45,0.178571,0.0600,-0.0208,0.0065,2.8592,-0.0026,-0.0051,0.19


In [40]:
mt5.symbol_info('BOVAS132W2')

SymbolInfo(custom=False, chart_mode=1, select=True, visible=True, session_deals=28, session_buy_orders=0, session_sell_orders=0, volume=25000, volumehigh=100000, volumelow=1, time=1752058093, digits=2, spread=2, spread_float=True, ticks_bookdepth=32, trade_calc_mode=35, trade_mode=4, start_time=0, expiration_time=1752278399, trade_stops_level=0, trade_freeze_level=0, trade_exemode=3, swap_mode=0, swap_rollover3days=3, margin_hedged_use_leg=False, expiration_mode=11, filling_mode=3, order_mode=127, order_gtc_mode=0, option_mode=0, option_right=1, bid=0.05, bidhigh=0.06, bidlow=0.04, ask=0.07, askhigh=0.88, asklow=0.05, last=0.06, lasthigh=0.08, lastlow=0.04, volume_real=25000.0, volumehigh_real=100000.0, volumelow_real=1.0, option_strike=132.0, point=0.01, trade_tick_value=0.01, trade_tick_value_profit=0.01, trade_tick_value_loss=0.01, trade_tick_size=0.01, trade_contract_size=1.0, trade_accrued_interest=0.0, trade_face_value=0.0, trade_liquidity_rate=0.0, volume_min=1.0, volume_max=176

In [31]:
mt5.symbol_select(ativo_base)

dados = mt5.copy_rates_from_pos(ativo_base, mt5.TIMEFRAME_D1, 0, 180)

# 4. Converte para DataFrame e ajusta datas
df = pd.DataFrame(dados)
df['time'] = pd.to_datetime(df['time'], unit='s')
print(df[['time', 'open', 'high', 'low', 'close', 'tick_volume']])

          time    open    high     low   close  tick_volume
0   2024-10-15  127.50  127.80  126.75  127.40        74336
1   2024-10-16  127.73  128.65  127.27  128.07        81708
2   2024-10-17  126.60  127.43  126.40  127.31        69809
3   2024-10-18  128.12  128.23  126.66  127.00       111994
4   2024-10-21  127.36  127.62  126.66  126.75       134049
..         ...     ...     ...     ...     ...          ...
175 2025-07-03  136.45  138.15  136.09  137.92        77130
176 2025-07-04  137.39  138.45  137.39  138.13        29900
177 2025-07-07  138.05  138.09  136.17  136.45        72051
178 2025-07-08  136.34  136.40  135.66  136.15        39229
179 2025-07-09  135.94  135.94  135.45  135.49          951

[180 rows x 6 columns]


In [ ]:
# LIMPA A MARKWT WATCH DO MT5

lista_remocao = list(grade_opcoes.index)

for nome in lista_remocao:
    mt5.symbol_select(nome,False)

mt5.symbol_select(ativo_base,False)

    

True

In [ ]:

mt5.shutdown()


True